In [174]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

### load the data

In [175]:
train_data = pd.read_csv("../minicomp-rossman/data/train.csv")

/Users/sinarampe/opt/anaconda3/envs/mini_comp/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### some basic EDA and Datacleaning

In [176]:
train_data.head()

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,2013-01-01,1115.0,2.0,0.0,0.0,0.0,0.0,a,1.0
1,2013-01-01,379.0,2.0,0.0,0.0,0.0,0.0,a,1.0
2,2013-01-01,378.0,2.0,0.0,0.0,0.0,0.0,a,1.0
3,2013-01-01,377.0,2.0,0.0,0.0,0.0,0.0,a,1.0
4,2013-01-01,376.0,2.0,0.0,0.0,0.0,0.0,a,1.0


In [177]:
train_data.shape

(637774, 9)

##### Zero sales days are ignored in scoring - the pipeline should look for these rows and drop them

In [178]:
train_data = train_data[train_data["Sales"]!=0]
train_data = train_data.dropna(subset = ["Sales"])

##### The Test Data doest have a Customer-Column, so we drop it. The Weekday is encoded in the Date column, so we drop it.

In [179]:
train_data = train_data.drop(labels=["Customers", "DayOfWeek"], axis=1)

##### convert Date column from str to dt, then create Y, m, d-column

In [180]:
train_data['Date'] = pd.to_datetime(train_data['Date'])
train_data["day"] = train_data['Date'].map(lambda x: x.day)
train_data["month"] = train_data['Date'].map(lambda x: x.month)
train_data["year"] =  train_data['Date'].map(lambda x: x.year)- 2013
train_data = train_data.drop(labels="Date", axis=1)

##### drop rows with nan values

In [181]:
train_data = train_data.dropna(subset = ["Store", "Open", "Promo", "StateHoliday", "SchoolHoliday"])

##### StateHoliday column has "0" as str and 0.0 as float -> convert all to str

In [182]:
train_data["StateHoliday"].replace({0.0: "0"}, inplace=True)
train_data["StateHoliday"].unique()

array(['a', '0', 'b', 'c'], dtype=object)

In [183]:
print(train_data.shape)
train_data.isna().sum()

(439929, 9)


Store            0
Sales            0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
day              0
month            0
year             0
dtype: int64

In [184]:
##### understand the data with Pandas Profiler

In [123]:
# profile = ProfileReport(train_data)

In [124]:
# profile

### Initial baseline modelling - just taking the mean

In [185]:
train = train_data.iloc[:int(0.8*train_data.shape[0]), :]
test = train_data.iloc[int(0.8*train_data.shape[0]):, :]

x_train = train.drop(labels="Sales", axis=1)
y_train = train.loc[:, "Sales"]
x_test = test.drop(labels="Sales", axis=1)
y_test = test.loc[:, "Sales"]

In [186]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# defining evaluation metric
def compute_rmse(actual, prediction):
    """
    Computs RMSE (root mean squared error) between predictions from a model
    and the actual values of the target variable.
    """
    
    rmse = sqrt(mean_squared_error(actual, prediction))
    
    # rounding to 2 decimal places
    print('RMSE is ', round(rmse,2))
    
    return rmse

In [187]:
lazy_estimator_predictions = pd.DataFrame(y_test.copy())

# using mean of entire training set
lazy_estimator_predictions.loc[:,'predicted_sales'] = y_train.mean()
lazy_estimator_predictions.head().round()

,Sales,predicted_sales
508072,7189.0,6779.0
508073,8793.0,6779.0
508074,10151.0,6779.0
508075,9479.0,6779.0
508076,7084.0,6779.0


In [192]:
lazy_estimator_rmse = compute_rmse(y_test, lazy_estimator_predictions.loc[:, 'predicted_sales']) 
print("mean absolute percantage error is " + str(lazy_estimator_rmse / y_test.mean()))

RMSE is  3105.15
mean absolute percantage error is 0.43928673483994946


### Linear Regression model

In [194]:
# one-hot-encode StateHoliday Column
import category_encoders as ce

ce_one = ce.OneHotEncoder(cols=["StateHoliday"]) 
ce_one_train = ce_one.fit_transform(x_train)
ce_one_test = ce_one.transform(x_test)

In [195]:
ce_one_train.head()

,Store,Open,Promo,StateHoliday_1,StateHoliday_2,StateHoliday_3,StateHoliday_4,SchoolHoliday,day,month,year
27,353.0,1.0,0.0,1,0,0,0,1.0,1,1,0
115,335.0,1.0,0.0,1,0,0,0,1.0,1,1,0
147,512.0,1.0,0.0,1,0,0,0,1.0,1,1,0
162,494.0,1.0,0.0,1,0,0,0,1.0,1,1,0
199,530.0,1.0,0.0,1,0,0,0,1.0,1,1,0


In [196]:
from sklearn.linear_model import LinearRegression

linear_regression_model = LinearRegression()
linear_regression_model.fit(ce_one_train, y_train)
linear_regression_predictions = linear_regression_model.predict(ce_one_test)

In [198]:
linear_regression_rmse = compute_rmse(y_test, linear_regression_predictions) 
print("mean absolute percantage error is " + str(linear_regression_rmse / y_test.mean()))

RMSE is  2828.98
mean absolute percantage error is 0.4002163175187697


### Make predictions on Holdout_data and create a submission.csv file

In [165]:
holdout_data = pd.read_csv("../minicomp-rossman/data/holdout_b29.csv")

/Users/sinarampe/opt/anaconda3/envs/mini_comp/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [166]:
holdout_data.head()

,Unnamed: 0,Date,Store,DayOfWeek,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,0,2014-08-01,371,5,550,1,1,0,1
1,1,2014-08-01,372,5,815,1,1,0,1
2,2,2014-08-01,373,5,357,1,1,0,1
3,3,2014-08-01,380,5,1512,1,1,0,1
4,4,2014-08-01,374,5,878,1,1,0,1


In [167]:
holdout_data = holdout_data.rename(columns={"Unnamed: 0": "Id"})
holdout_data["Id"] = holdout_data["Id"] + 1
holdout_data.head()

,Id,Date,Store,DayOfWeek,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,2014-08-01,371,5,550,1,1,0,1
1,2,2014-08-01,372,5,815,1,1,0,1
2,3,2014-08-01,373,5,357,1,1,0,1
3,4,2014-08-01,380,5,1512,1,1,0,1
4,5,2014-08-01,374,5,878,1,1,0,1


In [146]:
holdout_data['Date'] = pd.to_datetime(holdout_data['Date'])
holdout_data["day"] = holdout_data['Date'].map(lambda x: x.day)
holdout_data["month"] = holdout_data['Date'].map(lambda x: x.month)
holdout_data["year"] =  holdout_data['Date'].map(lambda x: x.year)- 2013
holdout_data = holdout_data.drop(labels="Date", axis=1)
holdout_data["StateHoliday"].replace({0.0: "0"}, inplace=True)
holdout_data_prep = holdout_data.drop(labels=["Customers", "DayOfWeek", "Id"], axis=1)

ce_one_holdout = ce_one.transform(holdout_data_prep)
ce_one_holdout.head()

,Store,Open,Promo,StateHoliday_1,StateHoliday_2,StateHoliday_3,StateHoliday_4,SchoolHoliday,day,month,year
0,371,1,1,0,1,0,0,1,1,8,1
1,372,1,1,0,1,0,0,1,1,8,1
2,373,1,1,0,1,0,0,1,1,8,1
3,380,1,1,0,1,0,0,1,1,8,1
4,374,1,1,0,1,0,0,1,1,8,1


In [147]:
holdout_regression_predictions = linear_regression_model.predict(ce_one_holdout)

In [149]:
output = pd.DataFrame({'Id': holdout_data.Id, 'Sales': holdout_regression_predictions})
output.to_csv('Submission.csv', index=False)

In [199]:
sub = pd.read_csv("Submission.csv")
print(sub.head())
sub.shape

   Id        Sales
0   1  8712.527180
1   2  8712.643430
2   3  8712.759680
3   4  8713.573431
4   5  8712.875930


(315540, 2)